### Query 작성

- gender_and_age = man, woman, boy, girl
- hair = long hair, short hair  # 턱선 위로는 짧은 머리, 아래로는 긴 머리
- top_color = red, orange, yellow, green, khaki, light blue, blue, dark blue, purple, pink, maroon, white, beige, black, brown
- top_type = short sleeve shirt, long sleeve shirt, coat, winter coat
- bottom_color = red, orange, yellow, green, khaki, light blue, blue, dark blue, purple, pink, maroon, white, beige, black, brown
- bottom type = short pants, long pants, skirt  (dress도 skirt로 처리)
- bag = backpack, bag

In [79]:
# ===== 토글 선택지 구현
togleList = [
    ["man", "woman", "boy", "girl"],
     ["long hair", "short hair"],
      ["none", "red", "orange", "yellow", "green", "khaki", "light blue", "blue", "dark blue", "purple", "pink", "maroon", "white", "beige", "black", "brown"],
       ["none", "short sleeve shirt", "long sleeve shirt", "coat", "winter coat"],
        ["none", "red", "orange", "yellow", "green", "khaki", "light blue", "blue", "dark blue", "purple", "pink", "maroon", "white", "beige", "black", "brown"],
         ["none", "short pants", "long pants", "skirt"],
          ["none", "backpack", "bag"]
    ]

In [80]:
togleSelect = [1, 1, 5, 4, 8, 1, 0]

In [81]:
selected = []

for i in range(len(togleList)):
    selected_value = togleList[i][togleSelect[i]]
    selected.append(selected_value)

gender_and_age, hair, top_color, top_type, bottom_color, bottom_type, bag = selected
query = ""

In [82]:
if gender_and_age == "man" or gender_and_age == "boy":
    gender = "He"
else:
    gender = "She"

In [83]:
# 상하의 인상착의 정보가 전혀 존재하지 않는 경우는 해당 부분에 대한 쿼리 작성을 생략, 예외처리 진행하였습니다.
if top_type == "none" and bottom_type == "none":
    raise ValueError("인상착의 정보가 존재하지 않습니다.")


# 상하의 색상 정보 없는 경우
elif top_color == "none" and bottom_color == "none":
    if bottom_type == "skirt":
        query = f"A {gender_and_age} wearing a {top_type} and a {bottom_type}. "
    else:
        query = f"A {gender_and_age} wearing a {top_type} and {bottom_type}. "


# 상하의 정보 중 단 하나를 알고 있으며, 색상 정보를 제외하고 종류만 알고 있는 경우
elif top_color == "none" and bottom_type == "none":
    query = f"A {gender_and_age} wearing a {top_type}. "

elif bottom_color == "none" and top_type == "none":
    if bottom_type == "skirt":
        query = f"A {gender_and_age} wearing a {bottom_type}. "
    else:
        query = f"A {gender_and_age} wearing {bottom_type}. "


# 상하의 정보 중 하나의 색상 정보만 누락된 경우
elif top_color == "none":
    if bottom_type == "skirt":
        query = f"A {gender_and_age} wearing a {top_type} and a {bottom_color} {bottom_type}. "
    else:
        query = f"A {gender_and_age} wearing a {top_type} and {bottom_color} {bottom_type}. "

elif bottom_color == "none":
    if bottom_type == "skirt":
        query = f"A {gender_and_age} wearing a {top_color} {top_type} and a {bottom_type}. "
    else:
        query = f"A {gender_and_age} wearing a {top_color} {top_type} and {bottom_type}. "


# 상하의 정보 중 하나의 종류 정보가 누락된 경우
elif top_type == "none":
    if bottom_type == "skirt":
        query = f"A {gender_and_age} wearing a {bottom_color} {bottom_type}. "
    else:
        query = f"A {gender_and_age} wearing {bottom_color} {bottom_type}. "

elif bottom_type == "none":
    query = f"A {gender_and_age} wearing a {top_color} {top_type}. "

# 모든 정보가 포함된 경우
else:
    if bottom_type == "skirt":
        query = f"A {gender_and_age} wearing a {top_color} {top_type} and a {bottom_color} {bottom_type}. "
    else:
        query = f"A {gender_and_age} wearing a {top_color} {top_type} and {bottom_color} {bottom_type}. "

In [84]:
# 손에 드는 건 carrying이 미세하게 더 나은 결과를 보임 (holding ↔ carrying)
if bag == "backpack":
    query += f"{gender} has {hair}. {gender} is carrying a {bag}."
elif bag == "bag":
    query += f"{gender} has {hair}. {gender} is holding a {bag}."
elif bag == "none":
    query += f"{gender} has {hair}."

In [85]:
print(query)

A woman wearing a khaki winter coat and dark blue short pants. She has short hair.


### 번역

In [86]:
!pip install openai

In [87]:
import openai

OPENAI_API_KEY = ""  # openAI-API 키를 넣으세요
openai.api_key = OPENAI_API_KEY

def translate_english_to_korean(text):
    response = openai.chat.completions.create(
    model = "gpt-3.5-turbo",
    presence_penalty = 0.9,
    messages = [
            {"role": "system", "content": """
            You are a translator.
            I will provide you with the information of the character's description in English, so please translate this information into Korean correctly.

            An example is as follows.

            Case 1.
            Input sentence: A woman wearing a blue long sleeve shirt and white short pants. She has long hair. She is holding a bag.
            Desired Translation: 긴 머리. 파란색 긴팔 셔츠와 흰 반바지를 입고, 가방을 들고 있음.

            Case 2.
            Input sentence: A boy wearing a maroon long sleeve shirt and black long pants. He has long hair.
            Desired Translation: 긴 머리. 자주색 긴팔 셔츠와 검정 긴 바지를 입고 있음.

            Case 3.
            Input sentence: A girl wearing a red long sleeve shirt and a dark blue skirt. She has short hair. She is carrying a backpack.
            Desired Translation: 짧은 머리. 빨간 긴팔 셔츠와 남색 치마를 입고, 가방을 메고 있음.

            Case 4.
            Input sentence: A man wearing a orange winter coat and blue long pants. He has short hair. He is carrying a backpack.
            Desired Translation: 짧은 머리. 주황색 패딩과 파란색 긴 바지를 입고, 가방을 메고 있음.

            Case 5.
            Input sentence: A girl wearing a purple short sleeve shirt and pink short pants. She has short hair. She is holding a backpack.
            Desired Translation: 짧은 머리. 보라색 반팔 셔츠와 핑크색 반바지를 입고, 가방을 들고 있음.

            Case 6.
            Input sentence: A man wearing a coat and long pants. He has short hair.
            Desired Translation: 짧은 머리. 코트와 긴 바지를 입고 있음.

            Case 7.
            Input sentence: A boy wearing a winter coat. He has long hair. He is carrying a backpack.
            Desired Translation: 긴 머리. 패딩을 입고, 가방을 메고 있음.

            Case 8.
            Input sentence: A woman wearing a skirt. She has short hair. She is holding a backpack.
            Desired Translation: 짧은 머리. 치마를 입고, 가방을 들고 있음.

            Case 9.
            Input sentence: A man wearing a coat and khaki short pants. He has long hair.
            Desired Translation: 긴 머리. 코트와 카키색 반바지를 입고 있음.

            Case 10.
            Input sentence: A girl wearing a light blue winter coat and a skirt. She has short hair. "
            Desired Translation: 짧은 머리. 하늘색 패딩과 치마를 입고 있음.

            Case 11.
            Input sentence: A boy wearing dark blue short pants. He has short hair.
            Desired Translation: 짧은 머리. 남색 반바지를 입고 있음.

            Case 12.
            Input sentence: A woman wearing a beige coat. She has short hair. She is holding a backpack."
            Desired Translation: 짧은 머리. 베이지색 코트를 입고, 가방을 들고 있음.

            Please respond to every sentence simply.
            """},
            {"role": "user", "content": text},
    ]
    )

    return response.choices[0].message.content

In [88]:
english_text = query
korean_text = translate_english_to_korean(english_text)
print("Eng query: ", english_text)
print("Kor query: ", korean_text)

Eng query:  A woman wearing a khaki winter coat and dark blue short pants. She has short hair.
Kor query:  짧은 머리. 카키색 패딩과 남색 반바지를 입고 있음.
